In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install requests BeautifulSoup4 playwright
!pip install fastapi uvicorn pydantic nest_asyncio
!playwright install
# Install Ollama on Colab
!curl -fsSL https://ollama.com/install.sh | sh

# Start Ollama server in background
import subprocess
import threading
import time

def start_ollama_server():
    subprocess.run(["ollama", "serve"], check=True)

# Start server in background thread
server_thread = threading.Thread(target=start_ollama_server, daemon=True)
server_thread.start()
time.sleep(10)  # Wait for server to start

# Pull the model you want
!ollama pull mistral

!pip install firebase-admin

import firebase_admin
from firebase_admin import credentials, db
cred = credentials.Certificate('/content/drive/MyDrive/content/bookflowautomation.json')

firebase_admin.initialize_app(cred, {
    'databaseURL': 'https://bookflowautomation-default-rtdb.firebaseio.com/'
})

Playwright Host validation warning: 
╔══════════════════════════════════════════════════════╗
║ Host system is missing dependencies to run browsers. ║
║ Missing libraries:                                   ║
║     libwoff2dec.so.1.0.2                             ║
║     libgstgl-1.0.so.0                                ║
║     libgstcodecparsers-1.0.so.0                      ║
║     libavif.so.13                                    ║
║     libharfbuzz-icu.so.0                             ║
║     libenchant-2.so.2                                ║
║     libsecret-1.so.0                                 ║
║     libhyphen.so.0                                   ║
║     libmanette-0.2.so.0                              ║
╚══════════════════════════════════════════════════════╝
    at validateDependenciesLinux (/usr/local/lib/python3.11/dist-packages/playwright/driver/package/lib/server/registry/dependencies.js:269:9)
    at process.processTicksAndRejections (node:internal/process/task_queues:105

Exception in thread Thread-13 (start_ollama_server):
Traceback (most recent call last):
  File "/usr/lib/python3.11/threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.11/threading.py", line 982, in run
    self._target(*self._args, **self._kwargs)
  File "/tmp/ipython-input-11-1478753188.py", line 13, in start_ollama_server
  File "/usr/lib/python3.11/subprocess.py", line 571, in run
    raise CalledProcessError(retcode, process.args,
subprocess.CalledProcessError: Command '['ollama', 'serve']' returned non-zero exit status 1.


ValueError: The default Firebase app already exists. This means you called initialize_app() more than once without providing an app name as the second argument. In most cases you only need to call initialize_app() once. But if you do want to initialize multiple apps, pass a second argument to initialize_app() to give each app a unique name.

In [ ]:
!ollama pull mistral

In [ ]:
!ollama list

NAME              ID              SIZE      MODIFIED      
mistral:latest    3944fe81ec14    4.1 GB    2 seconds ago    


In [ ]:
import os
import json
import asyncio
from datetime import datetime
import requests
from bs4 import BeautifulSoup
from playwright.async_api import async_playwright
# --- Settings ---
URL = "https://en.wikisource.org/wiki/The_Gates_of_Morning/Book_1/Chapter_1"
CHAPTER_ID = "chapter_1"
OUTPUT_DIR = "output"
SCREENSHOT_DIR = os.path.join(OUTPUT_DIR, "screenshots")
SCREENSHOT_PATH = os.path.join(SCREENSHOT_DIR, f"{CHAPTER_ID}.png")
JSON_PATH = os.path.join(OUTPUT_DIR, f"{CHAPTER_ID}.json")

# --- Ensure output dirs exist ---
os.makedirs(SCREENSHOT_DIR, exist_ok=True)

# --- Step 1: Scrape text content ---
def fetch_chapter_text(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    content_div = soup.find("div", {"class": "mw-parser-output"})
    paragraphs = content_div.find_all(["p", "h2", "h3"])
    text = "\n\n".join(p.get_text(strip=True) for p in paragraphs if p.get_text(strip=True))

    title = soup.find("h1").get_text(strip=True)

    return {
        "book": "The Gates of Morning",
        "chapter": title,
        "url": url,
        "content": text,
        "timestamp": datetime.utcnow().isoformat() + "Z"
    }

# --- Step 2: Take Screenshot with Playwright (async) ---
async def capture_screenshot(url, screenshot_path):
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()

        await page.goto(url)
        await page.wait_for_selector(".mw-parser-output")  # Wait for page to load

        # Get full page height and width
        total_height = await page.evaluate("document.body.scrollHeight")
        total_width = await page.evaluate("document.body.scrollWidth")
        await page.set_viewport_size({"width": total_width, "height": total_height})

        # Take full page screenshot
        await page.screenshot(path=screenshot_path, full_page=True)
        await browser.close()

# --- Step 3: Save Data ---
def save_output(data, json_path):
    with open(json_path, 'w', encoding='utf-8') as f:
        json.dump(data, f, indent=2, ensure_ascii=False)

# --- Run async main ---
async def main():
    print("[*] Fetching chapter content...")
    chapter_data = fetch_chapter_text(URL)
    chapter_data["screenshot_path"] = SCREENSHOT_PATH

    print("[*] Saving JSON...")
    save_output(chapter_data, JSON_PATH)

    print("[*] Taking screenshot with Playwright...")
    await capture_screenshot(URL, SCREENSHOT_PATH)

    print(f"[✓] Done! JSON saved to: {JSON_PATH}")
    print(f"[✓] Screenshot saved to: {SCREENSHOT_PATH}")


In [ ]:
import subprocess
import time
import json
import os
import asyncio

def run_ollama(prompt: str, model="mistral") -> str:
    """
    Sends a prompt to Ollama and returns the response from the specified model.
    """
    print(f"\n🧠 Prompting {model}...\n")
    try:
        result = subprocess.run(
            ["ollama", "run", model],
            input=prompt.encode(),
            capture_output=True,
            check=True
        )
        output = result.stdout.decode().strip()
        return output
    except subprocess.CalledProcessError as e:
        print("❌ Error running Ollama:", e.stderr.decode())
        return ""


def ai_writer(chapter_text: str) -> str:
    prompt = (
        f"Rewrite the following chapter to improve clarity, grammar, and style, "
        f"but preserve the meaning and tone:\n\n---\n{chapter_text}\n---\n"
    )
    return run_ollama(prompt)


def ai_reviewer(spun_text: str) -> str:
    prompt = (
        f"Review the following rewritten chapter for quality, coherence, grammar, "
        f"and originality. Provide detailed feedback and any necessary improvements:\n\n"
        f"{spun_text}\n"
    )
    return run_ollama(prompt)


async def iterative_refinement(original_text: str, max_cycles=3):
    print("\n🚀 Starting Writer + Reviewer refinement loop...\n")
    current_version = original_text

    for cycle in range(1, max_cycles + 1):
        print(f"\n🔁 Cycle {cycle} - AI Writer generating new version...\n")
        spun = await asyncio.to_thread(ai_writer, current_version)

        print(f"\n📋 Cycle {cycle} - AI Reviewer feedback...\n")
        review = await asyncio.to_thread(ai_reviewer, spun)

        print(f"\n✅ AI Review Summary (Cycle {cycle}):\n{review}\n")
        current_version = spun

        time.sleep(1)

    print("\n✅ Final Output after refinement:\n")
    print(current_version)
    return current_version


def load_chapter_content(json_path: str) -> dict:
    """
    Loads chapter JSON and returns a dictionary.
    """
    if not os.path.exists(json_path):
        raise FileNotFoundError(f"❌ File not found: {json_path}")

    with open(json_path, "r", encoding="utf-8") as f:
        data = json.load(f)
        return data


In [ ]:
# api_server.py (Run in Google Colab cell)

from fastapi import FastAPI
from pydantic import BaseModel
import uvicorn
import nest_asyncio
import asyncio
import uuid
import os
# Colab fix for asyncio loop
nest_asyncio.apply()

# FastAPI app
from fastapi.middleware.cors import CORSMiddleware

app = FastAPI()

# ✅ Add CORS Middleware to allow requests from your React frontend
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],  # You can later restrict this to ["http://localhost:3000"]
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)
from fastapi import Query

@app.get("/search")
def search_chapter(query: str = Query(...)):
    results = chapter_collection.query(
        query_texts=[query],
        n_results=3
    )
    return results

# Create folders
os.makedirs("output/screenshots", exist_ok=True)

class GenerateRequest(BaseModel):
    url: str
    max_cycles: int = 1

status_storage = {}

@app.post("/generate")
async def generate_chapter(req: GenerateRequest):
    chapter_id = f"chapter_{uuid.uuid4().hex[:6]}"
    screenshot_path = f"output/screenshots/{chapter_id}.png"
    json_path = f"output/{chapter_id}.json"

    status_storage[chapter_id] = {"status": "scraping", "stage": "🔍 Scraping content from URL..."}

    # Step 1: Scrape content
    chapter_data = fetch_chapter_text(req.url)
    chapter_data["screenshot_path"] = screenshot_path
    save_output(chapter_data, json_path)

    status_storage[chapter_id] = {"status": "screenshot", "stage": "📸 Taking screenshot..."}

    # Step 2: Screenshot (async Playwright)
    await capture_screenshot(req.url, screenshot_path)

    status_storage[chapter_id] = {"status": "ai_processing", "stage": f"🤖 AI processing ({req.max_cycles} cycles)..."}

    # Step 3: AI refinement
    original_text = chapter_data["content"]
    ai_version = await iterative_refinement(original_text, max_cycles=req.max_cycles)

    # ✅ Save to Firebase
    chapter_ref = db.reference(f"chapters/{chapter_id}")
    chapter_ref.set({
        "chapter_id": chapter_id,
        "chapter": chapter_data["chapter"],
        "book": chapter_data["book"],
        "url": chapter_data["url"],
        "content": chapter_data["content"],
        "ai_version": ai_version,
        "timestamp": chapter_data["timestamp"],
        "screenshot_path": screenshot_path,
        "status": "draft"
    })

    return {
        "title": chapter_data.get("chapter", "Untitled"),
        "original": original_text,
        "ai_version": ai_version
    }

 # Saves data to ./chromadb folder
@app.get("/chapters")
def get_all_chapters():
    ref = db.reference("chapters")
    return ref.get()
import threading


def run_server():
    uvicorn.run(app, host="0.0.0.0", port=8000)

server_thread = threading.Thread(target=run_server)
server_thread.start()


INFO:     Started server process [348]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
ERROR:    [Errno 98] error while attempting to bind on address ('0.0.0.0', 8000): address already in use
INFO:     Waiting for application shutdown.


In [ ]:
!pip install pyngrok
from pyngrok import ngrok
ngrok.set_auth_token("2yDkBg6GzWVhmOXSddCKSd2NhDH_6qHxuRKfS9QeRiscHHVyt")
public_url = ngrok.connect(8000)
print(f"Your public URL: {public_url}")


ERROR:asyncio:Task exception was never retrieved
future: <Task finished name='Task-34' coro=<Server.serve() done, defined at /usr/local/lib/python3.11/dist-packages/uvicorn/server.py:68> exception=SystemExit(1)>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/uvicorn/server.py", line 163, in startup
    server = await loop.create_server(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/asyncio/base_events.py", line 1536, in create_server
    raise OSError(err.errno, msg) from None
OSError: [Errno 98] error while attempting to bind on address ('0.0.0.0', 8000): address already in use

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/lib/python3.11/threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.11/threading.py", line 982, in run
    self._target(*self._args, **self._kwargs)
  File "/tmp/ipython-input-16-2123626462.py", line 98, in

Your public URL: NgrokTunnel: "https://fe22-34-16-133-110.ngrok-free.app" -> "http://localhost:8000"
